<a href="https://colab.research.google.com/github/pati-dev/text-multi-label/blob/master/attributes_from_text.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Boot

In [0]:
%matplotlib inline
import pandas as pd

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


Load data

In [3]:
bizDF = pd.read_csv("/content/drive/My Drive/Search Project/toronto_filtered_reviews_all_cat.csv", encoding='latin1')
bizDF.head()

,Unnamed: 0,business_id,categories,user_id,text,RestaurantsPriceRange2,BikeParking,RestaurantsTakeOut,GoodForKids,OutdoorSeating,RestaurantsGoodForGroups,RestaurantsReservations,RestaurantsDelivery,WiFi
0,1,c35qzLN6ItDpVIoj2uQW4Q,"Soup, Restaurants, Sandwiches",h9y55WNNg7SYg3kQwzTMmQ,"Had the soup and sandwich. Wow, it was great....",2,True,True,True,True,True,False,True,u'free'
1,2,c35qzLN6ItDpVIoj2uQW4Q,"Soup, Restaurants, Sandwiches",QfCfgc5PyNVwd_DgPzmeuA,I can't remember when I had a lunch as good as...,2,True,True,True,True,True,False,True,u'free'
2,3,c35qzLN6ItDpVIoj2uQW4Q,"Soup, Restaurants, Sandwiches",mKemb3MpSyUbsVUVSmI5tg,wow! the wraps are hearty and healthy. i had t...,2,True,True,True,True,True,False,True,u'free'
3,4,c35qzLN6ItDpVIoj2uQW4Q,"Soup, Restaurants, Sandwiches",zbSBmoujGgIPuNWQcny14g,First time here today w/ a male friend who had...,2,True,True,True,True,True,False,True,u'free'
4,5,c35qzLN6ItDpVIoj2uQW4Q,"Soup, Restaurants, Sandwiches",XDYnGRhaCQteZNFPZonRow,I don't even think my own mother could make so...,2,True,True,True,True,True,False,True,u'free'


In [4]:
bizDF.columns

Index(['Unnamed: 0', 'business_id', 'categories', 'user_id', 'text',
       'RestaurantsPriceRange2', 'BikeParking', 'RestaurantsTakeOut',
       'GoodForKids', 'OutdoorSeating', 'RestaurantsGoodForGroups',
       'RestaurantsReservations', 'RestaurantsDelivery', 'WiFi'],
      dtype='object')

We have multiple rows for each *business* $\times$ *user* combination.

In [11]:
print(len(bizDF.business_id.unique()))
print(len(bizDF))
print(len(bizDF.text.unique()))

888
97916
97766


Merge all reviews for a restaurant separated by a *space*.

In [62]:
def combineText(reviews):
  return " ".join(reviews)

reviewsDF = pd.DataFrame(bizDF.groupby(['business_id'])['text'].apply(combineText).reset_index())
print(reviewsDF.head())
print(len(reviewsDF))

              business_id                                               text
0  -9u2uFwhlj3Yq4mG5IweoA  I'm not sure what to think of this place...hip...
1  -BbnAc9YEO6pjvJGEtFbVQ  Lovely little place for ramen amongst the many...
2  -CLTZ0cO-ooKDDmXHFoGMg  This was a planned visit as we were visiting a...
3  -EX_qttyhm5YNb8NWtAcCQ  Fun little bar for some country music!!! I wou...
4  -ICGmF2qUVKdvOehVNgPbg  Lamesa was one of those wonderful unexpected e...
888


Merge reviews DF with shrunk down version of biz DF.

In [59]:
features = ['RestaurantsPriceRange2',
            'BikeParking', 
            'RestaurantsTakeOut',
            'GoodForKids', 
            'OutdoorSeating', 
            'RestaurantsGoodForGroups',
            'RestaurantsReservations', 
            'RestaurantsDelivery', 
            'WiFi']
featuresDF = bizDF.loc[:, features + ['business_id']].drop_duplicates()

finalDF = reviewsDF.merge(featuresDF, on='business_id')

888